In [9]:
import os
import pandas as pd
from binance.client import Client
from binance.exceptions import BinanceAPIException

# Replace 'apikey' and 'secret' with your Binance API key and secret
apikey = 'your_api_key'
secret = 'your_api_secret'

# Initialize the Binance client
client = Client(apikey, secret)

# Ask for the coin symbol as input
coin_symbol = input("Enter the coin symbol (e.g., BNB): ").upper()  # Convert to uppercase

# Define the trading pair symbols for 'BTCUSDT' and the chosen coin
btcusdt_trading_pair = 'BTCUSDT'
coin_trading_pair = f'{coin_symbol}USDT'

# Fetch 1-minute candle data for 'BTCUSDT' and the chosen coin
interval = Client.KLINE_INTERVAL_1MINUTE
historical_btcusdt = []
historical_coin = []
end_time = None
while len(historical_btcusdt) < 1000:
    new_data_btcusdt = client.get_historical_klines(symbol=btcusdt_trading_pair, interval=interval, end_str=end_time, limit=1000)
    new_data_coin = client.get_historical_klines(symbol=coin_trading_pair, interval=interval, end_str=end_time, limit=1000)
    
    if not new_data_btcusdt or not new_data_coin:
        break
    
    historical_btcusdt.extend(new_data_btcusdt)
    historical_coin.extend(new_data_coin)
    end_time = int(new_data_btcusdt[0][0]) - 1  # Use 'BTCUSDT' Open Time

# Create DataFrames from the historical data
columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume',
           'Number of Trades', 'TB Base Volume', 'TB Quote Volume']
hist_df_btcusdt = pd.DataFrame(historical_btcusdt, columns=columns)
hist_df_coin = pd.DataFrame(historical_coin, columns=columns)

# Preprocess the data
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
for df in [hist_df_btcusdt, hist_df_coin]:
    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

# Create a Pandas Excel writer using XlsxWriter engine
output_path = r'D:\Projects\Binance\Crypto Data'
os.makedirs(output_path, exist_ok=True)
output_file = os.path.join(output_path, f'{coin_symbol}_BTCUSDT_1Mint_500000.xlsx')

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    hist_df_btcusdt.to_excel(writer, sheet_name='BTCUSDT', index=False)
    hist_df_coin.to_excel(writer, sheet_name=coin_symbol, index=False)

print(f"Historical data for '{coin_symbol}' and 'BTCUSDT' saved to '{output_file}'.")

Enter the coin symbol (e.g., BNB): ada


ValueError: 11 columns passed, passed data had 12 columns